This script parses movements from the CSV file produced across the whole posterior. Think carefully about how you want to class an "introduction" because if you want a conservative measure (i.e. once a branch has entered the location of interest, it cannot leave) you will need to parse the whole tree as this does not keep track of clades.

To generate the input file here, use the BEAST software called  TreeMarkovJumpHistoryAnalyzer (command in same folder in repo). Make sure you remove burnin in at that stage because we are assuming it has been removed for these commands

In [1]:
import csv
from collections import defaultdict
import tqdm
import numpy as np


In [20]:
markov_file = ""


In [15]:
def parse_jumps(markov_file):

    all_tree_movements = defaultdict(list)

    with open(markov_file) as f:
        data = csv.DictReader(f)
        for l in tqdm.tqdm(data):
            tree = l['treeId']
            if l['startLocation'] != l['endLocation']: #they should all be different but just to check
                all_tree_movements[tree].append((l['startLocation'],l['endLocation'],l['time']))
                
                    
    return all_tree_movements

In [13]:
all_tree_movements = parse_jumps(test_file)

987126it [00:02, 342248.68it/s]


In [23]:
location_of_interest = ["Connecticut"]

introductions = {}

for tree, movements in all_tree_movements.items():
    introductions[tree] = 0
    for pair in movements:
        end_loc = pair[1]
        start_loc = pair[0]
        
        if end_loc in location_of_interest and start_loc not in location_of_interest:
            introductions[tree] += 1     

In [26]:
average_num_introductions = np.mean(list(introductions.values()))
print(average_num_introductions)

In [34]:
difference = {}
for tree, num_intros in introductions.items():
    difference[tree] = abs(num_intros - average_num_introductions)
    
average_tree = min(difference, key=difference.get)
print(average_tree)

10000000
